# Step 1: Installation and Setup

In [ ]:
# Installing TensorFlow
! pip install -q tensorflow-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 40.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.1.4 which is incompatible.
tensorflow 2.9.2 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.11.0 which is incompatible.
tensorflow 2.9.2 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.11.0 which is incompatible.
tensorflow 2.9.2 requires tensorflow-estimator<2.10.0,>=2.9.0rc0, but you have tensorflow-estimator 2.11.0 which is incompatible.


In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.11.0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Step 2: Importing the dataset from Kaggle

In [ ]:
# install Kaggle API
! pip install -q kaggle

In [ ]:
# create a directory as kaggle
! mkdir -p ~/.kaggle

In [ ]:
# importing kaggle API key to Google Colab
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# copy API key to kaggle directory
! cp kaggle.json ~/.kaggle

In [ ]:
# disable the API key
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# import the dataset
! kaggle datasets download -d tongpython/cat-and-dog #this is the API command 
#of the dataset.

 99% 216M/218M [00:10<00:00, 24.2MB/s]
100% 218M/218M [00:10<00:00, 21.9MB/s]


In [ ]:
# unzipping the dataset
! unzip -q /content/cat-and-dog.zip

In [ ]:
training_dir = '/content/training_set/training_set'
test_dir = '/content/test_set/test_set'

# Step 3: Building the model

In [ ]:
# reshape the images
img_shape = (128,128,3) #128*128 with 3 filters R,G,B.
#pre-trained model expects this image format.

# Loading the Pre-Trained Model (MobileNetV2)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, include_top=False, weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [ ]:
# freezing the model
base_model.trainable = False
#if we don't freeze the model and train the model then initial weights
#also change. We train only custom layers and not the whole network.

# Defining the custom head for network

In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
#we are changing the shape as it is not suited by output layer of custom head.
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
# output layer / prediction layer
#for binary classification units = 1 and activation function =sigmoid.
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

# Define the transfer learning model

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [ ]:
# compile the model
opt = tf.keras.optimizers.RMSprop(lr=0.0001)

In [ ]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Create Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_test = ImageDataGenerator(rescale=1/255.0)
#rescaling

In [ ]:
train_generator = data_gen_train.flow_from_directory(directory=training_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 8005 images belonging to 2 classes.


In [ ]:
test_generator = data_gen_test.flow_from_directory(directory=test_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2023 images belonging to 2 classes.


# Step 4: Training the model

In [ ]:
model.fit(train_generator, epochs=5, validation_data=test_generator)

Epoch 1/5
63/63 [==============================] - 51s 666ms/step - loss: 0.1863 - accuracy: 0.9327 - val_loss: 0.8260 - val_accuracy: 0.9436
Epoch 2/5
63/63 [==============================] - 29s 465ms/step - loss: 0.0792 - accuracy: 0.9699 - val_loss: 4.6608 - val_accuracy: 0.7469
Epoch 3/5
63/63 [==============================] - 28s 436ms/step - loss: 0.0501 - accuracy: 0.9836 - val_loss: 3.5970 - val_accuracy: 0.7776
Epoch 4/5
63/63 [==============================] - 29s 464ms/step - loss: 0.0334 - accuracy: 0.9904 - val_loss: 6.0074 - val_accuracy: 0.6426
Epoch 5/5
63/63 [==============================] - 29s 455ms/step - loss: 0.0212 - accuracy: 0.9933 - val_loss: 3.3117 - val_accuracy: 0.7336


# Step 5: Fine Tuning

In [ ]:
base_model.trainable = True

In [ ]:
len(base_model.layers)

154

In [ ]:
fine_tune_at = 100

In [ ]:
# freeze the layers before 100
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
# compile the model #binary classification problem.
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model
model.fit_generator(train_generator, epochs=5, validation_data=test_generator)

<ipython-input-58-83abb010d82d>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=5, validation_data=test_generator)


Epoch 1/5
63/63 [==============================] - 33s 474ms/step - loss: 0.0202 - accuracy: 0.9941 - val_loss: 5.3205 - val_accuracy: 0.5803
Epoch 2/5
63/63 [==============================] - 29s 455ms/step - loss: 0.0204 - accuracy: 0.9933 - val_loss: 2.8147 - val_accuracy: 0.7578
Epoch 3/5
63/63 [==============================] - 29s 453ms/step - loss: 0.0145 - accuracy: 0.9955 - val_loss: 4.5632 - val_accuracy: 0.5966
Epoch 4/5
63/63 [==============================] - 33s 521ms/step - loss: 0.0105 - accuracy: 0.9973 - val_loss: 1.5202 - val_accuracy: 0.7983
Epoch 5/5
63/63 [==============================] - 28s 448ms/step - loss: 0.0106 - accuracy: 0.9959 - val_loss: 0.6795 - val_accuracy: 0.8695


In [59]:
model.evaluate_generator(test_generator,steps=50) #testing

<ipython-input-59-0a887b631832>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator,steps=50)


[0.6795240640640259, 0.8695007562637329]

In [61]:
#1st is loss and 2nd is accuracy.
#or#
model.evaluate(test_generator,steps=50)

16/50 [========>.....................] - ETA: 13s - loss: 0.6795 - accuracy: 0.8695

50/50 [==============================] - 7s 125ms/step - loss: 0.6795 - accuracy: 0.8695


[0.6795241236686707, 0.8695007562637329]

In [62]:
#1st is loss and 2nd is accuracy.